# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 27 2022 2:11PM,253846,10,8956001-D,"ACI Healthcare USA, Inc.",Released
1,Dec 27 2022 1:55PM,253849,10,Yusen-8956016,Yusen – 3D Matrix,Released
2,Dec 27 2022 1:55PM,253849,10,Yusen-8956017,Yusen – 3D Matrix,Released
3,Dec 27 2022 1:55PM,253849,10,Yusen-8956020,Yusen – 3D Matrix,Released
4,Dec 27 2022 1:55PM,253849,10,Yuen-8956021,Yusen – 3D Matrix,Released
5,Dec 27 2022 1:55PM,253849,10,Yusen-8956022,Yusen – 3D Matrix,Released
6,Dec 27 2022 1:55PM,253849,10,Yusen-8956023,Yusen – 3D Matrix,Released
7,Dec 27 2022 1:11PM,253847,16,EMERSA-8956014,"Emersa Waterbox, LLC",Released
8,Dec 27 2022 12:52PM,253844,19,ACG-2102492245,ACG North America LLC,Released
9,Dec 27 2022 12:52PM,253844,19,ACG-2102492246,ACG North America LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,253843,Released,1
37,253844,Released,2
38,253846,Released,1
39,253847,Released,1
40,253849,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253843,NaN,NaN,1.0
253844,NaN,NaN,2.0
253846,NaN,NaN,1.0
253847,NaN,NaN,1.0
253849,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253784,12.0,16.0,1.0
253788,0.0,0.0,1.0
253789,0.0,0.0,1.0
253790,0.0,3.0,2.0
253794,0.0,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253784,12,16,1
253788,0,0,1
253789,0,0,1
253790,0,3,2
253794,0,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253784,12,16,1
1,253788,0,0,1
2,253789,0,0,1
3,253790,0,3,2
4,253794,0,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253784,12,16,1
1,253788,,,1
2,253789,,,1
3,253790,,3,2
4,253794,,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 27 2022 2:11PM,253846,10,"ACI Healthcare USA, Inc."
1,Dec 27 2022 1:55PM,253849,10,Yusen – 3D Matrix
7,Dec 27 2022 1:11PM,253847,16,"Emersa Waterbox, LLC"
8,Dec 27 2022 12:52PM,253844,19,ACG North America LLC
10,Dec 27 2022 12:48PM,253833,10,"Methapharm, Inc."
25,Dec 27 2022 12:44PM,253843,10,Ivaoes Animal Health
26,Dec 27 2022 12:42PM,253842,10,"Nextsource Biotechnology, LLC"
27,Dec 27 2022 12:32PM,253841,19,"GCH Granules USA, Inc."
28,Dec 27 2022 12:23PM,253830,10,ISDIN Corporation
77,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 27 2022 2:11PM,253846,10,"ACI Healthcare USA, Inc.",,,1
1,Dec 27 2022 1:55PM,253849,10,Yusen – 3D Matrix,,,6
2,Dec 27 2022 1:11PM,253847,16,"Emersa Waterbox, LLC",,,1
3,Dec 27 2022 12:52PM,253844,19,ACG North America LLC,,,2
4,Dec 27 2022 12:48PM,253833,10,"Methapharm, Inc.",,,15
5,Dec 27 2022 12:44PM,253843,10,Ivaoes Animal Health,,,1
6,Dec 27 2022 12:42PM,253842,10,"Nextsource Biotechnology, LLC",,,1
7,Dec 27 2022 12:32PM,253841,19,"GCH Granules USA, Inc.",,,1
8,Dec 27 2022 12:23PM,253830,10,ISDIN Corporation,,,49
9,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation,,,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 27 2022 2:11PM,253846,10,"ACI Healthcare USA, Inc.",1,,
1,Dec 27 2022 1:55PM,253849,10,Yusen – 3D Matrix,6,,
2,Dec 27 2022 1:11PM,253847,16,"Emersa Waterbox, LLC",1,,
3,Dec 27 2022 12:52PM,253844,19,ACG North America LLC,2,,
4,Dec 27 2022 12:48PM,253833,10,"Methapharm, Inc.",15,,
5,Dec 27 2022 12:44PM,253843,10,Ivaoes Animal Health,1,,
6,Dec 27 2022 12:42PM,253842,10,"Nextsource Biotechnology, LLC",1,,
7,Dec 27 2022 12:32PM,253841,19,"GCH Granules USA, Inc.",1,,
8,Dec 27 2022 12:23PM,253830,10,ISDIN Corporation,49,,
9,Dec 27 2022 12:09PM,253832,10,ISDIN Corporation,22,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 27 2022 2:11PM,253846,10,"ACI Healthcare USA, Inc.",1,,
1,Dec 27 2022 1:55PM,253849,10,Yusen – 3D Matrix,6,,
2,Dec 27 2022 1:11PM,253847,16,"Emersa Waterbox, LLC",1,,
3,Dec 27 2022 12:52PM,253844,19,ACG North America LLC,2,,
4,Dec 27 2022 12:48PM,253833,10,"Methapharm, Inc.",15,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 27 2022 2:11PM,253846,10,"ACI Healthcare USA, Inc.",1,NaN,NaN
1,Dec 27 2022 1:55PM,253849,10,Yusen – 3D Matrix,6,NaN,NaN
2,Dec 27 2022 1:11PM,253847,16,"Emersa Waterbox, LLC",1,NaN,NaN
3,Dec 27 2022 12:52PM,253844,19,ACG North America LLC,2,NaN,NaN
4,Dec 27 2022 12:48PM,253833,10,"Methapharm, Inc.",15,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 27 2022 2:11PM,253846,10,"ACI Healthcare USA, Inc.",1,0.0,0.0
1,Dec 27 2022 1:55PM,253849,10,Yusen – 3D Matrix,6,0.0,0.0
2,Dec 27 2022 1:11PM,253847,16,"Emersa Waterbox, LLC",1,0.0,0.0
3,Dec 27 2022 12:52PM,253844,19,ACG North America LLC,2,0.0,0.0
4,Dec 27 2022 12:48PM,253833,10,"Methapharm, Inc.",15,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7360652,517,3.0,0.0
16,507653,2,0.0,0.0
19,1269139,18,4.0,0.0
20,253784,1,16.0,12.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7360652,517,3.0,0.0
1,16,507653,2,0.0,0.0
2,19,1269139,18,4.0,0.0
3,20,253784,1,16.0,12.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,517,3.0,0.0
1,16,2,0.0,0.0
2,19,18,4.0,0.0
3,20,1,16.0,12.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,517.0
1,16,Released,2.0
2,19,Released,18.0
3,20,Released,1.0
4,10,Executing,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Completed,0.0,0.0,0.0,12.0
Executing,3.0,0.0,4.0,16.0
Released,517.0,2.0,18.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Completed,0.0,0.0,0.0,12.0
1,Executing,3.0,0.0,4.0,16.0
2,Released,517.0,2.0,18.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Completed,0.0,0.0,0.0,12.0
1,Executing,3.0,0.0,4.0,16.0
2,Released,517.0,2.0,18.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()